In [25]:
directory = 'E:/capJC/paper/part1'

In [26]:
import os
import pandas as pd
from deepface import DeepFace
import matplotlib.pyplot as plt

In [27]:
def process_files(directory):
    # Initialize lists to store extracted and predicted data
    extracted_data = []
    predicted_data = []

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            try:
                # Extract age, gender, and race from filename
                age, gender, race, _ = filename.split('_')
                gender = 'male' if gender == '0' else 'female'
                race_dict = {0: 'White', 1: 'Black', 2: 'Asian', 3: 'Indian', 4: 'Others'}
                race = race_dict[int(race)]

                # Append extracted data to the list
                extracted_data.append([filename, age, gender, race])

                # Analyze the image using DeepFace
                img_path = os.path.join(directory, filename)
                analysis = DeepFace.analyze(img_path, actions=['age', 'gender', 'race'])

                # Append predicted data to the list
                predicted_data.append([filename, analysis['age'], analysis['gender'], analysis['dominant_race']])
            except Exception as e:
                # Handle any exceptions (e.g., incorrect file format)
                extracted_data.append([filename, 'NaN', 'NaN', 'NaN'])
                predicted_data.append([filename, 'NaN', 'NaN', 'NaN'])
        else:
            # Skip non-jpg files
            continue

    # Convert lists to dataframes
    extracted_df = pd.DataFrame(extracted_data, columns=['Filename', 'Age', 'Gender', 'Race'])
    predicted_df = pd.DataFrame(predicted_data, columns=['Filename', 'Age', 'Gender', 'Race'])

    # Save the dataframes to CSV files
    extracted_df.to_csv('extracted_data.csv', index=False)
    predicted_df.to_csv('predicted_data.csv', index=False)

    return extracted_df, predicted_df

In [28]:
def compare_accuracy(extracted_df, predicted_df):
    # Comparing actual vs predicted
    total = len(extracted_df)
    correct_age = sum(extracted_df['Age'] == predicted_df['Age'])
    correct_gender = sum(extracted_df['Gender'] == predicted_df['Gender'])
    correct_race = sum(extracted_df['Race'] == predicted_df['Race'])

    age_accuracy = correct_age / total * 100
    gender_accuracy = correct_gender / total * 100
    race_accuracy = correct_race / total * 100

    return age_accuracy, gender_accuracy, race_accuracy

In [29]:
def plot_accuracy(age_accuracy, gender_accuracy, race_accuracy):
    # Plotting the accuracies in a bar graph
    labels = ['Age', 'Gender', 'Race']
    accuracies = [age_accuracy, gender_accuracy, race_accuracy]

    plt.bar(labels, accuracies, color=['blue', 'green', 'red'])
    plt.xlabel('Categories')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy of Actual vs Predicted')
    plt.show()

In [30]:
# Main function to execute the workflow

def main(directory):
    extracted_df, predicted_df = process_files(directory)
    age_accuracy, gender_accuracy, race_accuracy = compare_accuracy(extracted_df, predicted_df)
    plot_accuracy(age_accuracy, gender_accuracy, race_accuracy)
main(directory)    

Action: race:  67%|██████▋   | 2/3 [00:00<00:00,  3.72it/s]  